In [3]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

In [4]:
# Import
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [5]:
# Source Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Parse to html
html_parsed_result = BeautifulSoup(requests.get(url).text, 'html.parser')

# Parse wikipedia table
nb_tb = html_parsed_result.find('table', class_ = 'wikitable')

# Find rows
nb_rw = nb_tb.find_all('tr')

# extract 'Postcode', 'Borough', 'Neighbourhood' from wikipedia table
nb_info = []

# Append data to nb_info list
for row in nb_rw:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    nb_info.append(info)    
nb_info[0][-1] = 'Neighborhood'

# Create dataframe from nb_info list
nb_df = pd.DataFrame(nb_info[1:], columns=nb_info[0])

In [6]:
# Clean by replacing not assigned values in Neighborhood column to the same as Borough in the cell
nb_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
for i in nb_not_assigned:
    nb_df['Neighborhood'][i] = nb_df['Borough'][i]

In [7]:
# Clean by droping unassigned cells
boroughs_not_assigned = nb_df.index[nb_df['Borough'] == 'Not assigned']
neighborhoods_not_assigned = nb_df.index[nb_df['Neighborhood'] == 'Not assigned']
neighborhoods_borough_not_assigned = boroughs_not_assigned & neighborhoods_not_assigned
nb_df.drop(nb_df.index[boroughs_not_assigned], inplace=True)
nb_df.reset_index(drop=True, inplace=True)

# Print top 10
nb_df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [8]:
# Group same borough
gp = nb_df.groupby('Postcode')
gp_nb = gp['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
gp_boroughs = gp['Borough'].apply(lambda x: set(x).pop())
gp_df = pd.DataFrame(list(zip(gp_boroughs.index, gp_boroughs, gp_nb)))
gp_df.columns = ['Postcode', 'Borough', 'Neighborhood']


# Print top 10
gp_df.head(12)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
print('The DataFrame shape is', gp_df.shape)

The DataFrame shape is (103, 3)


In [10]:
gp_df.to_csv('Segmenting_Clustering_Neighborhoods_Toronto_sec1.csv',index=False)